## Import

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
import pandas as pd
import altair as alt

## Scrape

In [4]:
o_url = "https://www.espn.com/college-football/stats/team/_/table/passing/sort/yardsPerGame/dir/desc"

In [5]:
d_url = "https://www.espn.com/college-football/stats/team/_/view/defense"

In [55]:
def get_ranking(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    section = soup.find("div", class_="ResponsiveTable")
    table_list = section.find_all("table", class_="Table")
    assert len(table_list) == 2
    team_table = table_list[0]
    team_ranking = []
    for i, row in enumerate(team_table.find_all("tr")[2:]):
        d = {
            'rank': i+1,
            'name': row.text,
            'img': row.find_all("img")[0]['src'],
        }
        team_ranking.append(d)
    return pd.DataFrame(team_ranking)

In [58]:
o_df = get_ranking(o_url)

In [57]:
d_df = get_ranking(d_url)

## Transform

In [63]:
merged_df = o_df[["rank", "name"]].merge(d_df, on="name", how="inner", suffixes=["_o", "_d"])

In [65]:
merged_df['rank_diff'] = abs(merged_df.rank_o - merged_df.rank_d)

## Export

In [68]:
merged_df.sort_values("rank_diff", ascending=False)[[
    'name',
    'rank_o',
    'rank_d',
    'rank_diff',
]].to_csv("analysis.csv", index=False)